# Aprendizado Não Supervisionado  - Regras De Associação
Prof: Nielsen Rechia

### Autores: 
Roberto Coutinho  
Thais Galho


### Exercício Online Retail com Regras de Associação

* Dados do conjunto de dados em https://archive.ics.uci.edu/ml/datasets/online+retail

Tarefas:

* Encontrar, para um único país (ex Italia), regras usando o algoritmo apriori, como visto no exemplo no jupyter. 
* Utilizar medidas de suporte, confiança e lift
* Testar alguns valores 
* Definir possíveis melhores valores para tais medidas utilizadas 

Descrição dos arquivos:

* Entregar via BB o código python ou jupyter e o passo a passo realizado (ex, motivos das escolhas dos valores etc.) em um zip.


## Imports

In [1]:
#conda install mlxtend
#conda install mlxtend --channel conda-forge

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Leitura do arquivo

In [2]:
df = pd.read_excel('Online Retail.xlsx')
print len (df)
df.head()

541909


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Pré-processamento

### Filtragem pelo país Itália

In [3]:
df_italy = df.loc[df["Country"] == "Italy"] 

print len(df_italy)

df_italy.head(5)

803


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7214,537022,22791,T-LIGHT GLASS FLUTED ANTIQUE,12,2010-12-03 15:45:00,1.25,12725.0,Italy
7215,537022,21287,SCENTED VELVET LOUNGE CANDLE,12,2010-12-03 15:45:00,1.25,12725.0,Italy
7216,537022,79337,BLUE FLOCK GLASS CANDLEHOLDER,6,2010-12-03 15:45:00,1.65,12725.0,Italy
7217,537022,85111,SILVER GLITTER FLOWER VOTIVE HOLDER,12,2010-12-03 15:45:00,1.25,12725.0,Italy
7218,537022,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,6,2010-12-03 15:45:00,2.10,12725.0,Italy


### Definição dos data types

In [4]:
columns_to_filter = ["InvoiceNo","Description", "Quantity"]
df_italy = df_italy.loc[:, columns_to_filter]
df_italy['InvoiceNo'] = df_italy['InvoiceNo'].astype('str')
df_italy['Description'] = df_italy['Description'].astype('str')
df_italy['Quantity'] = df_italy['Quantity'].astype('int')
df_italy.head(5)

,InvoiceNo,Description,Quantity
7214,537022,T-LIGHT GLASS FLUTED ANTIQUE,12
7215,537022,SCENTED VELVET LOUNGE CANDLE,12
7216,537022,BLUE FLOCK GLASS CANDLEHOLDER,6
7217,537022,SILVER GLITTER FLOWER VOTIVE HOLDER,12
7218,537022,6 CHOCOLATE LOVE HEART T-LIGHTS,6


### Transformação de dados

##### Para aqueles produtos com quantidade maior que 0, seta o valor para 1, e ajusta o tipo de dados para Boolean

In [5]:
df_italy.loc[df_italy.Quantity > 0, "Quantity"] = 1
df_italy.head(5)

,InvoiceNo,Description,Quantity
7214,537022,T-LIGHT GLASS FLUTED ANTIQUE,1
7215,537022,SCENTED VELVET LOUNGE CANDLE,1
7216,537022,BLUE FLOCK GLASS CANDLEHOLDER,1
7217,537022,SILVER GLITTER FLOWER VOTIVE HOLDER,1
7218,537022,6 CHOCOLATE LOVE HEART T-LIGHTS,1


In [6]:
df_italy['Quantity'] = df_italy['Quantity'].astype('bool')
df_italy.head(5)

,InvoiceNo,Description,Quantity
7214,537022,T-LIGHT GLASS FLUTED ANTIQUE,True
7215,537022,SCENTED VELVET LOUNGE CANDLE,True
7216,537022,BLUE FLOCK GLASS CANDLEHOLDER,True
7217,537022,SILVER GLITTER FLOWER VOTIVE HOLDER,True
7218,537022,6 CHOCOLATE LOVE HEART T-LIGHTS,True


### Tranformação da tabela 

###### Reshape para tranformar cada InvoiceNo em linha e os produtos como colunas

In [7]:
df_italy = df_italy.pivot(index='InvoiceNo', columns='Description', values="Quantity")
df_italy.head(5)

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,...,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
537022,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
541115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
541703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Preenchimento de valores nulls

In [8]:
df_italy = df_italy.fillna(False)
df_italy.head(5)

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,...,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
537022,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
539752,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
541115,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
541703,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
542238,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Remoção da coluna index

In [9]:
df_italy.reset_index(drop = True, inplace = True)
df_italy.head(5)

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,...,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
0,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
df1 = pd.get_dummies(df_italy)
df1.head(10)

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,...,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
0,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Execução do algoritmo Apriori

In [11]:
#encontrando regras com no mínimo 5% de suporte
frequent_items = apriori(df1, min_support=0.02, use_colnames=True)
frequent_items.head(5)

,support,itemsets
0,0.072727,(ABC TREASURE BOOK BOX )
1,0.054545,(ADULT APRON APPLE DELIGHT)
2,0.036364,(ASSORTED COLOUR MINI CASES)
3,0.036364,(BAKING MOULD CHOCOLATE CUPCAKES)
4,0.072727,(BAKING SET 9 PIECE RETROSPOT )


## Encontrar Confiança maior que 0.05

In [12]:
#encontrar regras com lift maior que 1
#rules = association_rules(frequent_items, metric='lift', min_threshold=1)
rules = association_rules(frequent_items, metric='confidence', min_threshold=0.05)
#rules = association_rules(frequent_items, metric='confidence')
type(rules)
rules.shape
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(TOY TIDY SPACEBOY ),0.036364,0.090909,0.036364,1.0,11.000000,0.033058,inf
1,"(POPPY'S PLAYHOUSE BEDROOM , TOY TIDY SPACEBOY...",(TOY TIDY PINK POLKADOT),0.036364,0.090909,0.036364,1.0,11.000000,0.033058,inf
2,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(JUMBO STORAGE BAG SKULLS),0.036364,0.036364,0.036364,1.0,27.500000,0.035041,inf
3,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(RECYCLING BAG RETROSPOT ),0.036364,0.109091,0.036364,1.0,9.166667,0.032397,inf
4,"(TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY , ...",(POPPY'S PLAYHOUSE BEDROOM ),0.036364,0.036364,0.036364,1.0,27.500000,0.035041,inf


## Display as 10 primeiras regras com maior suporte

In [13]:
#10 primeiras regras com maior suporte
rules.sort_values('support', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3124761,(VINTAGE CREAM CAT FOOD CONTAINER),(VINTAGE CREAM DOG FOOD CONTAINER),0.127273,0.127273,0.109091,0.857143,6.734694,0.092893,6.109091
1893713,(SET OF 3 CAKE TINS PANTRY DESIGN ),(JAM MAKING SET WITH JARS),0.236364,0.200000,0.109091,0.461538,2.307692,0.061818,1.485714
1893712,(JAM MAKING SET WITH JARS),(SET OF 3 CAKE TINS PANTRY DESIGN ),0.200000,0.236364,0.109091,0.545455,2.307692,0.061818,1.680000
3124760,(VINTAGE CREAM DOG FOOD CONTAINER),(VINTAGE CREAM CAT FOOD CONTAINER),0.127273,0.127273,0.109091,0.857143,6.734694,0.092893,6.109091
3714198,(SET OF TEA COFFEE SUGAR TINS PANTRY),(VINTAGE CREAM DOG FOOD CONTAINER),0.145455,0.127273,0.109091,0.750000,5.892857,0.090579,3.490909
3714199,(VINTAGE CREAM DOG FOOD CONTAINER),(SET OF TEA COFFEE SUGAR TINS PANTRY),0.127273,0.145455,0.109091,0.857143,5.892857,0.090579,5.981818
336869,(BREAD BIN DINER STYLE IVORY),(SET OF 3 CAKE TINS PANTRY DESIGN ),0.145455,0.236364,0.090909,0.625000,2.644231,0.056529,2.036364
336868,(SET OF 3 CAKE TINS PANTRY DESIGN ),(BREAD BIN DINER STYLE IVORY),0.236364,0.145455,0.090909,0.384615,2.644231,0.056529,1.388636
3633931,(SET OF 3 CAKE TINS PANTRY DESIGN ),(MINT KITCHEN SCALES),0.236364,0.109091,0.090909,0.384615,3.525641,0.065124,1.447727
1728885,(VINTAGE CREAM CAT FOOD CONTAINER),"(SET OF TEA COFFEE SUGAR TINS PANTRY, VINTAGE ...",0.127273,0.109091,0.090909,0.714286,6.547619,0.077025,3.118182


## Display as 10 primeiras regras com maior confiança

In [14]:
#10 primeiras regras com maior confianca
rules.sort_values('confidence', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(TOY TIDY SPACEBOY ),0.036364,0.090909,0.036364,1.0,11.0,0.033058,inf
3326756,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, TOY ...","(CHILDRENS APRON APPLES DESIGN, FELTCRAFT DOLL...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326754,"(JUMBO BAG TOYS , LARGE CAKE TOWEL PINK SPOTS,...","(CHILDRENS APRON APPLES DESIGN, FELTCRAFT DOLL...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326753,"(JUMBO BAG TOYS , LARGE CAKE TOWEL PINK SPOTS,...","(TOY TIDY PINK POLKADOT, FELTCRAFT DOLL MOLLY,...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326752,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, POPP...","(TOY TIDY PINK POLKADOT, FELTCRAFT DOLL MOLLY,...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326751,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, TOY ...","(CHILDRENS APRON APPLES DESIGN, FELTCRAFT DOLL...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326750,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, POPP...","(TOY TIDY PINK POLKADOT, CHILDRENS APRON APPLE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326749,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, POPP...","(TOY TIDY PINK POLKADOT, CHILDRENS APRON APPLE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326748,"(FELTCRAFT DOLL MOLLY, LARGE CAKE TOWEL PINK S...","(JUMBO STORAGE BAG SUKI, JUMBO BAG WOODLAND AN...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326747,"(FELTCRAFT DOLL MOLLY, LARGE CAKE TOWEL PINK S...","(JUMBO STORAGE BAG SUKI, JUMBO BAG WOODLAND AN...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf


## Display as 10 primeiras regras com maior LIFT

In [15]:
#10 primeiras regras com maior lift
rules.sort_values('lift', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2528925,"(POPPY'S PLAYHOUSE KITCHEN, TOY TIDY SPACEBOY )","(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, TOY ...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104151,"(REGENCY CAKESTAND 3 TIER, VINTAGE DOILY TRAVE...","(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104157,(VINTAGE DOILY TRAVEL SEWING KIT),"(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
4819509,"(JUMBO BAG WOODLAND ANIMALS, CHILDRENS APRON A...","(TOY TIDY SPACEBOY , FELTCRAFT DOLL MOLLY, RE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104155,(SET OF 12 FAIRY CAKE BAKING CASES),"(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104154,(CLASSIC CHROME BICYCLE BELL ),"(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
4819510,"(TOY TIDY SPACEBOY , CHILDRENS APRON APPLES D...","(JUMBO BAG WOODLAND ANIMALS, FELTCRAFT DOLL MO...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104152,(HAND WARMER RED LOVE HEART),"(REGENCY CAKESTAND 3 TIER, JAM MAKING SET WITH...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104150,"(VINTAGE DOILY TRAVEL SEWING KIT, SET OF 12 FA...","(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104160,"(CHILDRENS APRON APPLES DESIGN, ROUND SNACK BO...",(JUMBO STORAGE BAG SKULLS),0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf


## selecionando regras com lift maior que 2, confianca maior que 0.6 e suporte maior que 0.2

In [16]:
rules[(rules['lift'] >= 2) & (rules['confidence'] >= 0.06) & (rules['support'] >= 0.01 )]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(TOY TIDY SPACEBOY ),0.036364,0.090909,0.036364,1.000000,11.000000,0.033058,inf
1,"(POPPY'S PLAYHOUSE BEDROOM , TOY TIDY SPACEBOY...",(TOY TIDY PINK POLKADOT),0.036364,0.090909,0.036364,1.000000,11.000000,0.033058,inf
2,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(JUMBO STORAGE BAG SKULLS),0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
3,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(RECYCLING BAG RETROSPOT ),0.036364,0.109091,0.036364,1.000000,9.166667,0.032397,inf
4,"(TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY , ...",(POPPY'S PLAYHOUSE BEDROOM ),0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
5,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(JUMBO STORAGE BAG SUKI),0.036364,0.072727,0.036364,1.000000,13.750000,0.033719,inf
6,"(POPPY'S PLAYHOUSE BEDROOM , RECYCLING BAG RET...","(TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY )",0.036364,0.090909,0.036364,1.000000,11.000000,0.033058,inf
7,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...","(TOY TIDY SPACEBOY , JUMBO STORAGE BAG SKULLS)",0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
8,"(POPPY'S PLAYHOUSE BEDROOM , TOY TIDY SPACEBOY...","(TOY TIDY PINK POLKADOT, JUMBO STORAGE BAG SKU...",0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
9,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...","(TOY TIDY SPACEBOY , RECYCLING BAG RETROSPOT )",0.036364,0.072727,0.036364,1.000000,13.750000,0.033719,inf


## Encontrar LIFT maior que 1.0

In [17]:
#encontrar regras com lift maior que 1
rules = association_rules(frequent_items, metric='lift', min_threshold=1)
type(rules)
rules.shape
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(TOY TIDY SPACEBOY ),0.036364,0.090909,0.036364,1.0,11.000000,0.033058,inf
1,"(POPPY'S PLAYHOUSE BEDROOM , TOY TIDY SPACEBOY...",(TOY TIDY PINK POLKADOT),0.036364,0.090909,0.036364,1.0,11.000000,0.033058,inf
2,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(JUMBO STORAGE BAG SKULLS),0.036364,0.036364,0.036364,1.0,27.500000,0.035041,inf
3,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(RECYCLING BAG RETROSPOT ),0.036364,0.109091,0.036364,1.0,9.166667,0.032397,inf
4,"(TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY , ...",(POPPY'S PLAYHOUSE BEDROOM ),0.036364,0.036364,0.036364,1.0,27.500000,0.035041,inf


## Display as 10 primeiras regras com maior suporte

In [18]:
#10 primeiras regras com maior suporte
rules.sort_values('support', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3124758,(VINTAGE CREAM DOG FOOD CONTAINER),(VINTAGE CREAM CAT FOOD CONTAINER),0.127273,0.127273,0.109091,0.857143,6.734694,0.092893,6.109091
1893711,(SET OF 3 CAKE TINS PANTRY DESIGN ),(JAM MAKING SET WITH JARS),0.236364,0.200000,0.109091,0.461538,2.307692,0.061818,1.485714
3714197,(VINTAGE CREAM DOG FOOD CONTAINER),(SET OF TEA COFFEE SUGAR TINS PANTRY),0.127273,0.145455,0.109091,0.857143,5.892857,0.090579,5.981818
3714196,(SET OF TEA COFFEE SUGAR TINS PANTRY),(VINTAGE CREAM DOG FOOD CONTAINER),0.145455,0.127273,0.109091,0.750000,5.892857,0.090579,3.490909
3124759,(VINTAGE CREAM CAT FOOD CONTAINER),(VINTAGE CREAM DOG FOOD CONTAINER),0.127273,0.127273,0.109091,0.857143,6.734694,0.092893,6.109091
1893710,(JAM MAKING SET WITH JARS),(SET OF 3 CAKE TINS PANTRY DESIGN ),0.200000,0.236364,0.109091,0.545455,2.307692,0.061818,1.680000
336868,(SET OF 3 CAKE TINS PANTRY DESIGN ),(BREAD BIN DINER STYLE IVORY),0.236364,0.145455,0.090909,0.384615,2.644231,0.056529,1.388636
1728883,(VINTAGE CREAM CAT FOOD CONTAINER),"(SET OF TEA COFFEE SUGAR TINS PANTRY, VINTAGE ...",0.127273,0.109091,0.090909,0.714286,6.547619,0.077025,3.118182
4369772,(JAM MAKING SET WITH JARS),(BREAD BIN DINER STYLE IVORY),0.200000,0.145455,0.090909,0.454545,3.125000,0.061818,1.566667
2389474,(SET OF TEA COFFEE SUGAR TINS PANTRY),(VINTAGE CREAM CAT FOOD CONTAINER),0.145455,0.127273,0.090909,0.625000,4.910714,0.072397,2.327273


## Display as 10 primeiras regras com maior confiança

In [19]:
#10 primeiras regras com maior confianca
rules.sort_values('confidence', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(TOY TIDY SPACEBOY ),0.036364,0.090909,0.036364,1.0,11.0,0.033058,inf
3326754,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, TOY ...","(CHILDRENS APRON APPLES DESIGN, FELTCRAFT DOLL...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326752,"(JUMBO BAG TOYS , LARGE CAKE TOWEL PINK SPOTS,...","(CHILDRENS APRON APPLES DESIGN, FELTCRAFT DOLL...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326751,"(JUMBO BAG TOYS , LARGE CAKE TOWEL PINK SPOTS,...","(TOY TIDY PINK POLKADOT, FELTCRAFT DOLL MOLLY,...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326750,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, POPP...","(TOY TIDY PINK POLKADOT, FELTCRAFT DOLL MOLLY,...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326749,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, TOY ...","(CHILDRENS APRON APPLES DESIGN, FELTCRAFT DOLL...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326748,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, POPP...","(TOY TIDY PINK POLKADOT, CHILDRENS APRON APPLE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326747,"(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, POPP...","(TOY TIDY PINK POLKADOT, CHILDRENS APRON APPLE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326746,"(FELTCRAFT DOLL MOLLY, LARGE CAKE TOWEL PINK S...","(JUMBO STORAGE BAG SUKI, JUMBO BAG WOODLAND AN...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3326745,"(FELTCRAFT DOLL MOLLY, LARGE CAKE TOWEL PINK S...","(JUMBO STORAGE BAG SUKI, JUMBO BAG WOODLAND AN...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf


## Display as 10 primeiras regras com maior lift

In [20]:
#10 primeiras regras com maior lift
rules.sort_values('lift', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2528923,"(POPPY'S PLAYHOUSE KITCHEN, TOY TIDY SPACEBOY )","(JUMBO BAG TOYS , JUMBO STORAGE BAG SUKI, TOY ...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104150,(HAND WARMER RED LOVE HEART),"(REGENCY CAKESTAND 3 TIER, JAM MAKING SET WITH...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
4819505,"(JUMBO BAG WOODLAND ANIMALS, CHILDRENS APRON A...","(TOY TIDY SPACEBOY , FELTCRAFT DOLL MOLLY, RE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
4819506,"(TOY TIDY SPACEBOY , CHILDRENS APRON APPLES D...","(JUMBO BAG WOODLAND ANIMALS, FELTCRAFT DOLL MO...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104155,(VINTAGE DOILY TRAVEL SEWING KIT),"(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104153,(SET OF 12 FAIRY CAKE BAKING CASES),"(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104152,(CLASSIC CHROME BICYCLE BELL ),"(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
4819508,"(FELTCRAFT DOLL MOLLY, JUMBO BAG TOYS )","(LARGE CAKE TOWEL PINK SPOTS, RECYCLING BAG RE...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104149,"(REGENCY CAKESTAND 3 TIER, VINTAGE DOILY TRAVE...","(HAND WARMER RED LOVE HEART, JAM MAKING SET WI...",0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf
3104159,"(CHILDRENS APRON APPLES DESIGN, ROUND SNACK BO...",(LARGE CAKE TOWEL PINK SPOTS),0.036364,0.036364,0.036364,1.0,27.5,0.035041,inf


## selecionando regras com lift maior que 2, confianca maior que 0.6 e suporte maior que 0.2

In [21]:
rules[(rules['lift'] >= 2) & (rules['confidence'] >= 0.06) & (rules['support'] >= 0.01 )]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(TOY TIDY SPACEBOY ),0.036364,0.090909,0.036364,1.000000,11.000000,0.033058,inf
1,"(POPPY'S PLAYHOUSE BEDROOM , TOY TIDY SPACEBOY...",(TOY TIDY PINK POLKADOT),0.036364,0.090909,0.036364,1.000000,11.000000,0.033058,inf
2,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(JUMBO STORAGE BAG SKULLS),0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
3,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(RECYCLING BAG RETROSPOT ),0.036364,0.109091,0.036364,1.000000,9.166667,0.032397,inf
4,"(TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY , ...",(POPPY'S PLAYHOUSE BEDROOM ),0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
5,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...",(JUMBO STORAGE BAG SUKI),0.036364,0.072727,0.036364,1.000000,13.750000,0.033719,inf
6,"(POPPY'S PLAYHOUSE BEDROOM , RECYCLING BAG RET...","(TOY TIDY PINK POLKADOT, TOY TIDY SPACEBOY )",0.036364,0.090909,0.036364,1.000000,11.000000,0.033058,inf
7,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...","(TOY TIDY SPACEBOY , JUMBO STORAGE BAG SKULLS)",0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
8,"(POPPY'S PLAYHOUSE BEDROOM , TOY TIDY SPACEBOY...","(TOY TIDY PINK POLKADOT, JUMBO STORAGE BAG SKU...",0.036364,0.036364,0.036364,1.000000,27.500000,0.035041,inf
9,"(TOY TIDY PINK POLKADOT, POPPY'S PLAYHOUSE BED...","(TOY TIDY SPACEBOY , RECYCLING BAG RETROSPOT )",0.036364,0.072727,0.036364,1.000000,13.750000,0.033719,inf
